Sample pipeline of cleaning data. Largely just finding duplicate columns and dealing with NaN values

In [2]:
#Packages used
import numpy as np
import pandas as pd

Load data into pandas data frames

In [3]:
df_data = pd.read_csv('data.csv')
df_ec = pd.read_csv('ec.csv')
df_votes = pd.read_csv('votes.csv')
df_data.describe()

,Fips,Precincts,Votes,Less Than High School Diploma,At Least High School Diploma,At Least Bachelors's Degree,Graduate Degree,School Enrollment,Median Earnings 2010,White (Not Latino) Population,...,Adult.obesity,Diabetes,Sexually.transmitted.infections,HIV.prevalence.rate,Uninsured,Unemployment,Violent.crime,Homicide.rate,Injury.deaths,Infant.mortality
count,3141.00000,3109.000000,3.109000e+03,3141.000000,3141.000000,3141.000000,3141.000000,3141.000000,3141.000000,3141.000000,...,3136.000000,3136.000000,2952.000000,2320.000000,3135.000000,3135.000000,2953.000000,1251.000000,2844.000000,1414.000000
mean,30406.52149,54.940495,4.178247e+04,16.874514,83.045893,19.013419,6.449682,74.953932,25463.387576,78.778908,...,0.305523,0.107103,354.269682,174.590733,0.179860,0.077045,257.055923,6.285803,76.112553,7.737553
std,15156.09339,172.926255,1.134574e+05,7.349563,7.533330,8.646941,3.849162,5.261211,5078.837873,19.625220,...,0.042426,0.022529,273.108568,229.263418,0.054765,0.027680,207.563047,4.659909,24.604878,2.511056
min,1001.00000,1.000000,6.400000e+01,0.700000,29.900000,3.700000,0.000000,0.000000,0.000000,2.500000,...,0.131000,0.033000,37.400000,11.000000,0.031000,0.008000,0.000000,0.610000,23.300000,2.600000
25%,18181.00000,11.000000,4.847000e+03,11.400000,78.400000,13.100000,4.000000,72.300000,22332.413650,67.700000,...,0.283000,0.092000,182.550000,59.800000,0.140000,0.058000,114.620000,3.195000,59.300000,6.100000
50%,29179.00000,20.000000,1.094800e+04,15.400000,84.600000,16.900000,5.300000,75.150000,24830.888890,86.350000,...,0.307000,0.106000,271.150000,102.500000,0.177000,0.075000,201.920000,5.080000,72.800000,7.200000
75%,45083.00000,38.000000,2.882800e+04,21.600000,88.600000,22.600000,7.700000,77.950000,27548.556510,94.300000,...,0.331000,0.122000,440.350000,204.250000,0.215000,0.093000,340.650000,8.105000,88.925000,9.100000
max,56045.00000,4988.000000,2.652072e+06,52.100000,99.300000,71.000000,40.600000,100.000000,56674.350795,99.200000,...,0.479000,0.194000,2754.400000,3764.100000,0.460000,0.283000,1989.540000,51.490000,320.900000,19.600000


Set indexes to 'Fips' for easier joins down the line

In [4]:
try:
    df_data.set_index('Fips', inplace=True)
except:
    print('df_data index is already set to Fips')
try: 
    df_votes.set_index('Fips', inplace=True)
except:
    print('df_votes index is already set to Fips')


#### Cleaning data.csv values

Checked NaN values in each column

In [5]:
print(df_data.isnull().sum(axis=0))

State                                                            0
ST                                                               1
County                                                           0
Precincts                                                       32
Votes                                                           32
Less Than High School Diploma                                    0
At Least High School Diploma                                     0
At Least Bachelors's Degree                                      0
Graduate Degree                                                  0
School Enrollment                                                0
Median Earnings 2010                                             0
White (Not Latino) Population                                    0
African American Population                                      0
Native American Population                                       0
Asian American Population                                     

Deleted all columns with more than 90 NaN values

In [6]:
df_data_temp1 = df_data.loc[:,(df_data.isnull().values.sum(axis=0) < 90)]
print(df_data_temp1.shape)

(3141, 44)


Removed 'ST' and 'Votes' columns because they are redundant

In [7]:
df_data_clean = df_data_temp1.drop(columns = ['ST','Votes'])

Droped duplicate columns: 'White_Asian','White (Not Latino) Population','African American Population','Asian American Population','Amerindian','Hispanic', and 'Other'

In [8]:
try:
    df_data_clean = df_data_clean.drop(columns = ['White_Asian','White (Not Latino) Population','African American Population','Asian American Population','Amerindian','Hispanic','Other'])
except:
    print('Columns already dropped')

#### Cleaning votes.csv data

Checked Column Names

In [9]:
list(df_votes)

['Democrats 08 (Votes)',
 'Democrats 12 (Votes)',
 'Republicans 08 (Votes)',
 'Republicans 12 (Votes)',
 'votes16_trumpd',
 'votes16_clintonh']

Changed votes16_ columns to match other vote columns in order to keep consistency and not have to see the name Trump or Clinton more than necessary

In [10]:
df_votes.rename(columns= \
          {'votes16_trumpd':'Republicans 16 (Votes)',
           'votes16_clintonh':'Democrats 16 (Votes)'}, inplace=True)

Augmented votes.csv dataframe with relative vote count for Democrats and Republicans in 08',12',16'

In [11]:
df_votes['Relative Vote Count 08 (Democrats)'] \
= (df_votes['Democrats 08 (Votes)']
   / (df_votes['Democrats 08 (Votes)']+df_votes['Republicans 08 (Votes)']))

df_votes['Relative Vote Count 08 (Republicans)'] \
= (df_votes['Republicans 08 (Votes)']
   / (df_votes['Democrats 08 (Votes)']+df_votes['Republicans 08 (Votes)']))
df_votes.head()

df_votes['Relative Vote Count 12 (Democrats)'] \
= (df_votes['Democrats 12 (Votes)']
   / (df_votes['Democrats 12 (Votes)']+df_votes['Republicans 12 (Votes)']))

df_votes['Relative Vote Count 12 (Republicans)'] \
= (df_votes['Republicans 12 (Votes)']
   / (df_votes['Democrats 12 (Votes)']+df_votes['Republicans 12 (Votes)']))

df_votes['Relative Vote Count 16 (Democrats)'] \
= (df_votes['Democrats 16 (Votes)']
   / (df_votes['Democrats 16 (Votes)']+df_votes['Republicans 16 (Votes)']))

df_votes['Relative Vote Count 16 (Republicans)'] \
= (df_votes['Republicans 16 (Votes)']
   / (df_votes['Democrats 16 (Votes)']+df_votes['Republicans 16 (Votes)']))

#### Cleaning votes.csv data

Since votes are the dependent variable remove all rows with missing votes values as they cannot be used in analysis

In [12]:
df_votes_clean = df_votes.loc[df_votes.isnull().sum(axis=1) < 1]
df_votes_clean.shape
df_votes_clean.head()

,Democrats 08 (Votes),Democrats 12 (Votes),Republicans 08 (Votes),Republicans 12 (Votes),Republicans 16 (Votes),Democrats 16 (Votes),Relative Vote Count 08 (Democrats),Relative Vote Count 08 (Republicans),Relative Vote Count 12 (Democrats),Relative Vote Count 12 (Republicans),Relative Vote Count 16 (Democrats),Relative Vote Count 16 (Republicans)
Fips,,,,,,,,,,,,
5043,2598.0,2630.0,3860.0,3887.0,3967.0,2364.0,0.402292,0.597708,0.403560,0.596440,0.373401,0.626599
5087,2144.0,2099.0,3972.0,4263.0,4917.0,1587.0,0.350556,0.649444,0.329928,0.670072,0.244004,0.755996
13159,1935.0,1845.0,3916.0,4136.0,4353.0,1544.0,0.330713,0.669287,0.308477,0.691523,0.261828,0.738172
8037,13191.0,12792.0,8181.0,9411.0,8153.0,12652.0,0.617209,0.382791,0.576138,0.423862,0.608123,0.391877
13091,2595.0,2442.0,5543.0,5214.0,5021.0,1836.0,0.318874,0.681126,0.318966,0.681034,0.267756,0.732244


Joined vote share values to data.csv dataframe.

In [13]:
df_vote_share = df_votes_clean[['Relative Vote Count 08 (Democrats)',
 'Relative Vote Count 08 (Republicans)',
 'Relative Vote Count 12 (Democrats)',
 'Relative Vote Count 12 (Republicans)',
 'Relative Vote Count 16 (Democrats)',
 'Relative Vote Count 16 (Republicans)']]

In [14]:
df_datavotes = df_data_clean.join(df_vote_share,how='inner')
df_datavotes.describe()

,Precincts,Less Than High School Diploma,At Least High School Diploma,At Least Bachelors's Degree,Graduate Degree,School Enrollment,Median Earnings 2010,Native American Population,Other Race or Races,Latino Population,...,Adult.obesity,Diabetes,Uninsured,Unemployment,Relative Vote Count 08 (Democrats),Relative Vote Count 08 (Republicans),Relative Vote Count 12 (Democrats),Relative Vote Count 12 (Republicans),Relative Vote Count 16 (Democrats),Relative Vote Count 16 (Republicans)
count,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,...,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000,3109.000000
mean,54.940495,16.908845,83.026825,19.000611,6.445915,75.012609,25435.949477,1.513573,1.564715,7.948906,...,0.305560,0.107336,0.179128,0.076891,0.422354,0.577646,0.392064,0.607936,0.332787,0.667213
std,172.926255,7.334157,7.480831,8.656472,3.855546,5.024879,5034.281018,6.116896,1.493233,13.010701,...,0.042483,0.022371,0.054056,0.027508,0.140379,0.140379,0.150092,0.150092,0.160898,0.160898
min,1.000000,0.700000,29.900000,3.700000,0.000000,38.000000,0.000000,0.000000,0.000000,0.000000,...,0.131000,0.033000,0.031000,0.008000,0.050314,0.065992,0.034722,0.074124,0.032468,0.042509
25%,11.000000,11.500000,78.400000,13.100000,4.000000,72.350000,22330.872700,0.150000,0.900000,1.450000,...,0.283000,0.092000,0.139000,0.058000,0.319120,0.486566,0.283617,0.515853,0.213356,0.578609
50%,20.000000,15.400000,84.600000,16.800000,5.300000,75.200000,24798.574605,0.300000,1.300000,3.000000,...,0.307000,0.106000,0.177000,0.075000,0.418352,0.581648,0.379378,0.620622,0.299223,0.700777
75%,38.000000,21.600000,88.500000,22.600000,7.700000,77.950000,27516.438415,0.650000,1.850000,7.900000,...,0.332000,0.122000,0.213000,0.093000,0.513434,0.680880,0.484147,0.716383,0.421391,0.786644
max,4988.000000,52.100000,99.300000,71.000000,40.600000,100.000000,56674.350795,85.900000,33.000000,97.150000,...,0.479000,0.194000,0.389000,0.283000,0.934008,0.949686,0.925876,0.965278,0.957491,0.967532


Take a look at our NaN values after we filtered the bulk of them out

In [15]:
df_datavotes.isnull().values.sum(axis=0)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Drop the 7 rows with NaN values

In [16]:
df = df_datavotes.dropna(0)
print(df.shape)
df.describe()

(3102, 41)


,Precincts,Less Than High School Diploma,At Least High School Diploma,At Least Bachelors's Degree,Graduate Degree,School Enrollment,Median Earnings 2010,Native American Population,Other Race or Races,Latino Population,...,Adult.obesity,Diabetes,Uninsured,Unemployment,Relative Vote Count 08 (Democrats),Relative Vote Count 08 (Republicans),Relative Vote Count 12 (Democrats),Relative Vote Count 12 (Republicans),Relative Vote Count 16 (Democrats),Relative Vote Count 16 (Republicans)
count,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,...,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000,3102.000000
mean,55.054481,16.921003,83.062879,19.003965,6.450193,75.011992,25429.577808,1.511896,1.566167,7.928353,...,0.305630,0.107349,0.179083,0.076951,0.422700,0.577300,0.392411,0.607589,0.333083,0.666917
std,173.104608,7.330226,7.362754,8.654018,3.856004,4.984572,5023.654512,6.120533,1.494227,12.967330,...,0.042476,0.022385,0.054095,0.027494,0.140130,0.140130,0.149881,0.149881,0.160732,0.160732
min,1.000000,0.700000,43.250000,3.700000,0.000000,38.000000,0.000000,0.000000,0.000000,0.000000,...,0.131000,0.033000,0.031000,0.008000,0.078539,0.065992,0.058276,0.074124,0.036765,0.042509
25%,11.000000,11.500000,78.400000,13.100000,4.000000,72.350000,22331.257937,0.150000,0.900000,1.450000,...,0.283000,0.092000,0.139000,0.058000,0.319785,0.486563,0.284126,0.515510,0.213543,0.578603
50%,20.000000,15.400000,84.600000,16.800000,5.300000,75.200000,24796.125715,0.300000,1.300000,3.000000,...,0.307000,0.106000,0.177000,0.075000,0.418485,0.581515,0.379603,0.620397,0.299314,0.700686
75%,38.000000,21.600000,88.500000,22.600000,7.700000,77.950000,27512.122778,0.650000,1.850000,7.850000,...,0.332000,0.122000,0.213000,0.093000,0.513437,0.680215,0.484490,0.715874,0.421397,0.786457
max,4988.000000,52.100000,99.300000,71.000000,40.600000,93.350000,56674.350795,85.900000,33.000000,97.150000,...,0.479000,0.194000,0.389000,0.283000,0.934008,0.921461,0.925876,0.941724,0.957491,0.963235


In [59]:
df_all = df_data_clean.join(df_votes_clean,how='inner')
df_all['Country'] = 'U.S.A.'
df_all.to_csv('usa_election_data.csv', sep='\t')

Data now has no NaN values and is considered ready for us in analysis.